In [1]:
import glob
import sys
import os
import enum
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.figsize'] = [14, 7]

In [2]:
import import_ipynb
import drivers
import prepare

importing Jupyter notebook from drivers.ipynb
importing Jupyter notebook from prepare.ipynb


In [3]:
nifty = prepare.MergedDf()

In [4]:
nifty.describe()

,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),P/E,P/B,Div Yield
count,5225.000000,5225.000000,5225.000000,5225.000000,5.225000e+03,5225.000000,5225.000000,5225.000000,5225.000000
mean,4931.530211,4964.677742,4892.778804,4929.321809,1.515437e+08,6250.129768,19.882100,3.539041,1.419129
std,3247.573752,3256.301398,3231.571643,3243.876880,1.191109e+08,4840.834747,4.159403,0.798190,0.400195
min,853.000000,877.000000,849.950000,854.200000,1.394931e+06,40.120000,10.680000,1.920000,0.590000
25%,1667.450000,1688.250000,1644.400000,1668.750000,6.926587e+07,2620.680000,17.010000,3.020000,1.160000
50%,4877.850000,4930.250000,4833.050000,4875.050000,1.305892e+08,5462.340000,19.940000,3.470000,1.320000
75%,7588.550000,7635.550000,7532.450000,7580.200000,1.900432e+08,8149.000000,22.660000,3.800000,1.540000
max,12274.900000,12293.900000,12252.750000,12271.800000,1.414837e+09,54081.530000,29.900000,6.550000,3.180000


In [5]:
nifty.loc['1999-01-04']

Open                      896.40
High                      905.45
Low                       895.75
Close                     897.80
Shares Traded        32224833.00
Turnover (Rs. Cr)         811.39
P/E                        11.72
P/B                         2.08
Div Yield                   1.81
Name: 1999-01-04 00:00:00, dtype: float64

In [6]:
# debt = DebtCorpus()
# print(debt.Deposit(datetime(2000, 1, 1), 100))
# print(debt.Withdraw(datetime(2010, 1, 1), 100))
# debt.Get(datetime(2020, 1, 1))


# vanilla strategy with params

# monthly_sip = 100
# default_exposure = 0.5
# green_pe = 15
# red_pe = 28

# Every month, invest monthly_sip * default_exposure in index and invest monthly_sip * (1 - default_exposure) in debt.
# If nifty pe > red_pe, pull out all money from index to debt.
# if nifty pe < green_pe, pull out all money from debt to index.

In [7]:
nifty.index[-1].to_pydatetime()

datetime.datetime(2019, 12, 31, 0, 0)

In [8]:
def EvaluateStrategy(df, params):
    print('params:', json.dumps(params.__dict__, indent=2))
    push_num_installments = int(params.push_num_installments)
    pull_num_installments = int(params.pull_num_installments)
    # strategy
    curr_month = -1
    e = drivers.EquityCorpus(df)
    d = drivers.DebtCorpus()
    total_invested = 0
    num_installments = 0
    size_installment = 0;
    for ind in df.index:
        if ind.month != curr_month:
            curr_month = ind.month
            index_sip = params.monthly_sip * params.default_exposure
            debt_sip = params.monthly_sip * (1 - params.default_exposure)
            current_pe = df['P/E'][ind]
            if (current_pe < params.green_pe):
                # we are in bear market.
                if (0 == num_installments):
                    debt_funds = d.Get(ind)
                    # print('debt_funds', debt_funds, ind)
                    size_installment = debt_funds / params.push_num_installments
                to_invest = size_installment if num_installments < params.push_num_installments else d.Get(ind)
                # print('to_invest', to_invest, size_installment, d.Get(ind))
                debt_sip -= to_invest
                index_sip += to_invest
                num_installments+=1
            elif (current_pe > params.red_pe):
                # we are in bull market
                equity_funds = e.Get(ind)
                if (0 == num_installments):
                    # print('equity_funds', equity_funds, ind)
                    size_installment = equity_funds / params.pull_num_installments
                to_redeem = min(size_installment, equity_funds)
                # print('to_redeem', to_redeem, size_installment, e.Get(ind), ind)
                index_sip -= to_redeem
                debt_sip += to_redeem
                num_installments+=1
            else:
                num_installments = 0
            assert abs(index_sip + debt_sip - params.monthly_sip) < 0.01,\
                'index_sip:' + str(index_sip) + ', debt_sip: ' + str(debt_sip) + ', monthly_sip:' + str(params.monthly_sip) 
            if (index_sip > 0):
                # print('deposit in equity', index_sip, ind)
                e.Deposit(ind, index_sip)
            elif (index_sip < 0):
                # print('withdraw from equity', index_sip, ind)
                e.Withdraw(ind, - index_sip)
            if (debt_sip > 0):
                d.Deposit(ind, debt_sip)
            elif (debt_sip < 0):
                d.Withdraw(ind, - debt_sip)
            total_invested += params.monthly_sip
    start_date = df.index[0].to_pydatetime()
    end_date = df.index[-1].to_pydatetime()
    # print('start-end', start_date, end_date)
    # print('total_invested', total_invested)
    # print('e.Get()', e.Get(end_date))
    # print('d.Get()', d.Get(end_date))
    returns = (e.Get(end_date) + d.Get(end_date)) / total_invested
    print('returns', returns)
    return returns

In [9]:
# # Debug Strategy
# params = drivers.Parameters(monthly_sip=100,
#                             default_exposure=0.0,
#                             green_pe=12,
#                             red_pe=22,
#                             pull_num_installments=2.0,
#                             push_num_installments=2.0)
# EvaluateStrategy(nifty, params)

In [10]:
# Always and only equity investor
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=1,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 1,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 4.4901087169736345


4.4901087169736345

In [11]:
# Always and only debt investor
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 2.326306709798203


2.326306709798203

In [12]:
# Mixed investor, no rebalancing.
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.5,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.5,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 3.4082077133859188


3.4082077133859188

In [13]:
# Mixed investor, with rebalancing.
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.5,
                            green_pe=18,
                            red_pe=28,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.5,
  "green_pe": 18,
  "red_pe": 28,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 4.791885658856569


4.791885658856569

In [ ]:
from scipy import optimize

def f(z, *params):
    de, gpe, rpe, pli, psi = z
    p = drivers.Parameters(monthly_sip=100,
                           default_exposure=de,
                           green_pe=gpe,
                           red_pe=rpe,
                           pull_num_installments=pli,
                           push_num_installments=psi)
    return - 1 * EvaluateStrategy(nifty, p)

rranges = ((0.0, 1.0), (12, 20), (22, 30), slice(2, 12), slice(2, 12))
resbrute = optimize.brute(f, rranges, args=None, full_output=True,
                              finish=optimize.fmin)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 2.0,
  "push_num_installments": 2.0
}
returns 6.344933967297349
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 2.0,
  "push_num_installments": 3.0
}
returns 4.790596078905681
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 2.0,
  "push_num_installments": 4.0
}
returns 4.093975435669332
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 2.0,
  "push_num_installments": 5.0
}
returns 3.7017785060345623
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 2.0,
  "push_num_installments": 6.0
}
returns 3.4510536597393013
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,

returns 3.248720584636596
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 6.0,
  "push_num_installments": 8.0
}
returns 3.126647126889621
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 6.0,
  "push_num_installments": 9.0
}
returns 3.0330387285712956
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 6.0,
  "push_num_installments": 10.0
}
returns 2.958994713272106
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 6.0,
  "push_num_installments": 11.0
}
returns 2.8989704054524164
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 7.0,
  "push_num_installments": 2.0
}
returns 5.923642221823916
params: {
  "monthly_sip": 100

returns 2.8635103058569737
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 11.0,
  "push_num_installments": 2.0
}
returns 5.5137089086687086
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 11.0,
  "push_num_installments": 3.0
}
returns 4.380833513250474
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 11.0,
  "push_num_installments": 4.0
}
returns 3.8407988139643834
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 11.0,
  "push_num_installments": 5.0
}
returns 3.525226953888094
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.0,
  "pull_num_installments": 11.0,
  "push_num_installments": 6.0
}
returns 3.318366113626973
params: {
  "monthly_sip":

returns 3.4582966381597
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 5.0,
  "push_num_installments": 7.0
}
returns 3.2841521137251344
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 5.0,
  "push_num_installments": 8.0
}
returns 3.15626314985798
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 5.0,
  "push_num_installments": 9.0
}
returns 3.058405469492423
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 5.0,
  "push_num_installments": 10.0
}
returns 2.9811345788645798
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 5.0,
  "push_num_installments": 1

returns 3.080219393889722
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 9.0,
  "push_num_installments": 9.0
}
returns 2.994093031001225
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 9.0,
  "push_num_installments": 10.0
}
returns 2.9256164323285354
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 9.0,
  "push_num_installments": 11.0
}
returns 2.869870779952096
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 10.0,
  "push_num_installments": 2.0
}
returns 5.5387967208151645
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 22.42105263157895,
  "pull_num_installments": 10.0,
  "push_num_installmen

...

returns 2.991514719488716
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.263157894736842,
  "pull_num_installments": 10.0,
  "push_num_installments": 10.0
}
returns 2.9244879136115567
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.263157894736842,
  "pull_num_installments": 10.0,
  "push_num_installments": 11.0
}
returns 2.869731436886963
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.263157894736842,
  "pull_num_installments": 11.0,
  "push_num_installments": 2.0
}
returns 5.351379798100176
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.263157894736842,
  "pull_num_installments": 11.0,
  "push_num_installments": 3.0
}
returns 4.327526179046583
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.263157894736842,
  "pull_num_installments": 11.0,
  "push_num_ins

returns 3.020789485017383
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 4.0,
  "push_num_installments": 11.0
}
returns 2.9547585160525704
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 5.0,
  "push_num_installments": 2.0
}
returns 6.157780171267634
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 5.0,
  "push_num_installments": 3.0
}
returns 4.742099751706763
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 5.0,
  "push_num_installments": 4.0
}
returns 4.0862055165779765
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 5.0,
  "push_num_installm

returns 3.904963480579464
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 9.0,
  "push_num_installments": 5.0
}
returns 3.5821861096418965
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 9.0,
  "push_num_installments": 6.0
}
returns 3.3689584225672147
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 9.0,
  "push_num_installments": 7.0
}
returns 3.2176115735385467
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 9.0,
  "push_num_installments": 8.0
}
returns 3.1046256939680266
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 23.684210526315788,
  "pull_num_installments": 9.0,
  "push_num_install

returns 3.070678512764742
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 3.0,
  "push_num_installments": 11.0
}
returns 2.9986487818158523
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 4.0,
  "push_num_installments": 2.0
}
returns 6.526967432356609
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 4.0,
  "push_num_installments": 3.0
}
returns 4.929384342558534
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 4.0,
  "push_num_installments": 4.0
}
returns 4.204603866013973
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 4.0,
  "push_num_installme

returns 2.906882211445909
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 8.0,
  "push_num_installments": 2.0
}
returns 5.687018253312434
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 8.0,
  "push_num_installments": 3.0
}
returns 4.511280491041568
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 8.0,
  "push_num_installments": 4.0
}
returns 3.944224327818435
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 8.0,
  "push_num_installments": 5.0
}
returns 3.610650699616486
params: {
  "monthly_sip": 100,
  "default_exposure": 0.0,
  "green_pe": 12.0,
  "red_pe": 24.105263157894736,
  "pull_num_installments": 8.0,
  "push_num_installment